# Segment and quantify puncta

Segment puncta and analyze their properties. Calculate puncta summaries per ROI (cell or nucleus) if the ROI segmentation is available.

## Requirements

- A folder with images to analyze (see [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb) for details)
- A json parameter file specified using [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb)


<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [1]:
#################################
#  Don't modify the code below  #
#################################

import json
import os
from punctatools.lib.segment import segment_puncta_batch
from punctatools.lib.quantify import quantify_batch
from punctatools.lib.utils import load_parameters, save_parameters

2022-03-04 11:38:27,977 [INFO] WRITING LOG OUTPUT TO /home/amedyukh/.cellpose/run.log


### Specify the parameters for segmentation

<hr style="height:0.5px;">

#### Option 1 (preferred)

Specify the parameter file to read the parameters (`parameter_file`)

To set up the parameters and generate the parameter file, run the notebook [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb)

<hr style="height:0.5px;">

#### Option 2

Specify the parameter values (see [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb) for more details).

`input_dir`: directory with images to analyze

`output_dir`: directory to save the result of the puncta analysis

`roi_segmentation`: Set to `True` if the ROI segmentation was done and the last channel of the images in the `input_dir` contains the segmentation mask

`puncta_channels`: list of channel indices, starting form 0, that should be used to segment puncta

`roi_segmentation`: Set to `True` if the ROI segmentation was done and the last channel of the images in the `input_dir` contains the segmentation mask

`minsize_um`: miminum target puncta size (microns); default is 0.2

`maxsize_um`: maximum target puncta size (microns); default is 2

`num_sigma`: number of sigma values for the LoG detector; default is 5

`threshold_detection`: threshold used by the LoG detector to exclude low intensity centers. Reduce this to detect blobs with lower intensities. Should be close to 0 and can be both positive and negative.

`overlap`: a value between 0 and 1; parameter used by the LoG detector to remove the smaller one of two overlapping blobs

`threshold_background`: threshold used to remove low intensity puncta centers, provided relative to the ROI background value.

`background_percentile`: Percentile (between 0 and 100) of image intensity inside ROI used to calculate the background value.

`global_background`: If `True`, the background value is calculated globally as the `global_background_percentile` of all ROI.

`global_background_percentile`: Percentile (between 0 and 100) of ROI background values used to calculate the global background value. 

`threshold_segmentation`: Threshold for puncta segmentation. The way the threshold is applied is determined by `segmentation_mode`. For mode 0, choose values in the order of 0.001; for mode 1, choose values in the order of 50; for mode 2, choose values in the order of 3. Reduce to detect more/larger puncta, increase to detect fewer/smaller puncta

`segmentation_mode`: 0, 1, or 2. Determines the mode how `threshold_segmentation` is applied; 0: apply absolute threshold in LoG space; 1: apply threshold relative to background in LoG space; 2: apply threshold relative to the background in image intensity space.

`remove_out_of_roi`: If True, puncta (parts) that extend beyond cells/nuclei will be removed. 

`maxrad_um`: maximum puncta radius in microns; used to remove large puncta. 

`n_jobs`: number of processes to run in parallel. 

`channel_names`: list of names for the channels in the original image

`puncta_segm_dir`: subdirectory to store segmented puncta masks (added as extra channel(s) to the input data)

`puncta_quant_dir`: subdirectory to store measurements for individual puncta

`roi_quant_dir`: subdirectory to store measurements for individual ROI (nuclei, cells)

In [2]:
parameter_file = 'parameters.json'

# input_dir = "../example_data/stacks"
# roi_segmentation = False
# output_dir = "../test_output/puncta_analysis"

# puncta_channels = [1, 2]

# minsize_um = 0.2  
# maxsize_um = 2  
# num_sigma = 5
# threshold_detection = [0.001, 0.0001] 
# overlap = 1

# threshold_background = [3, 1.5]
# background_percentile = 50
# global_background = False
# global_background_percentile = 95

# threshold_segmentation = [0.001, 0.0005]
# segmentation_mode = 0

# remove_out_of_roi = False
# maxrad_um = None

# n_jobs=8

# channel_names = ['ch0', 'ch1', 'ch3']
# puncta_segm_dir = 'puncta_segm'
# puncta_quant_dir = 'puncta_quants'
# roi_quant_dir = 'cell_quants'

<hr style="height:2px;">

## Processing

<hr style="height:2px;">


### The following code loads the parameters 

In [3]:
#################################
#  Don't modify the code below  #
#################################

param_keys = [
    'puncta_segm_dir',
    'puncta_quant_dir',
    'roi_quant_dir',
    'puncta_channels',
    'roi_segmentation',
    'minsize_um',
    'maxsize_um',
    'num_sigma',
    'overlap', 
    'threshold_detection', 
    'threshold_background', 
    'global_background', 
    'global_background_percentile', 
    'background_percentile', 
    'threshold_segmentation',
    'segmentation_mode',
    'remove_out_of_roi', 
    'maxrad_um', 
    'n_jobs',
    'channel_names' 
]
param_matches = dict(output_dir='puncta_analysis_dir')
kwargs = load_parameters(vars(), param_keys, param_matches)
if kwargs['roi_segmentation']:
    param_matches['input_dir'] = 'roi_segmentation_dir'
else:
    param_matches['input_dir'] = 'converted_data_dir'
kwargs = load_parameters(vars(), param_keys, param_matches)
os.makedirs(kwargs['output_dir'], exist_ok=True)
params = save_parameters(kwargs, 
                         os.path.join(kwargs['output_dir'], parameter_file.split('/')[-1]))

kwargs

{'puncta_segm_dir': 'puncta_segm',
 'puncta_quant_dir': 'puncta_quants',
 'roi_quant_dir': 'cell_quants',
 'puncta_channels': [1, 2],
 'roi_segmentation': True,
 'minsize_um': 0.2,
 'maxsize_um': 2,
 'num_sigma': 5,
 'overlap': 1,
 'threshold_detection': [0.001, 0.0001],
 'threshold_background': [2, 1.5],
 'global_background': False,
 'global_background_percentile': 95,
 'background_percentile': 50,
 'threshold_segmentation': [80, 50],
 'segmentation_mode': 1,
 'remove_out_of_roi': False,
 'maxrad_um': None,
 'n_jobs': 8,
 'channel_names': ['ch0', 'ch1', 'ch3'],
 'output_dir': '/research/sharedresources/cbi/common/Anna/codes/punctatools/test_output/puncta_analysis',
 'input_dir': '/research/sharedresources/cbi/common/Anna/codes/punctatools/test_output/roi'}

### The following code segments puncta in all image in the input folder

In [4]:
#################################
#  Don't modify the code below  #
#################################

channel_names = kwargs.pop('channel_names')
puncta_segm_dir = kwargs.pop('puncta_segm_dir')
roi_quant_dir = kwargs.pop('roi_quant_dir')
puncta_quant_dir = kwargs.pop('puncta_quant_dir')

segm_kwargs = kwargs.copy()
segm_kwargs['output_dir'] = os.path.join(segm_kwargs['output_dir'], puncta_segm_dir)
segment_puncta_batch(parallel=True, process_name='Segment puncta', **segm_kwargs)

Run Segment puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 129.98it/s]


Segment puncta done


### The following code quantifies all images

In [5]:
#################################
#  Don't modify the code below  #
#################################

quantify_batch(input_dir=segm_kwargs['output_dir'],
               output_dir_puncta=os.path.join(kwargs['output_dir'], puncta_quant_dir),
               output_dir_roi=os.path.join(kwargs['output_dir'], roi_quant_dir),
               parallel=True, n_jobs=kwargs['n_jobs'],
               channel_names=channel_names,
               puncta_channels=kwargs['puncta_channels'],
               process_name='Quantify puncta')

Run Quantify puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 156.24it/s]
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/punctatools/lib/quantify.py:124: RuntimeWarning: invalid value encountered in long_scalars
  coloc = np.sum((overlap[cur_roi_pix] > 0) * 1) / np.sum(union[cur_roi_pix])
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/punctatools/lib/quantify.py:124: RuntimeWarning: invalid value encountered in long_scalars
  coloc = np.sum((overlap[cur_roi_pix] > 0) * 1) / np.sum(union[cur_roi_pix])
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/research/sharedresources/cbi/public/conda_envs/puncta

Quantify puncta done


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 288.05it/s]
